<a href="https://colab.research.google.com/github/baileysmoko/Fabric/blob/main/cmc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================
# Step 1. Install dependencies
# ===============================
!pip install yfinance requests

# ===============================
# Step 2. Imports
# ===============================
import requests
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from google.colab import files
import time


In [ ]:
# ===============================
# Step 3. Setup your CMC API key
# ===============================
CMC_API_KEY = "b9d792ee-8769-44d9-ab43-109cd8973c29"  # ⬅️ replace with your real key
headers = {"X-CMC_PRO_API_KEY": CMC_API_KEY}

import requests
import pandas as pd

In [ ]:
# ===============================
# Step 4. Get Fear & Greed Index
# ===============================
def fetch_fear_greed_history(limit=500, start=None):
    url = "https://pro-api.coinmarketcap.com/v3/fear-and-greed/historical"
    params = {"limit": limit}
    if start is not None:
        params["start"] = start
    resp = requests.get(url, headers=headers, params=params).json()

    if "data" not in resp:
        raise ValueError(f"Unexpected response: {resp}")

    df = pd.DataFrame(resp["data"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
    df.rename(columns={"value": "fear_greed_value"}, inplace=True)
    return df[["timestamp", "fear_greed_value", "value_classification"]]


dfs = []
start = 1
chunk_size = 500

while True:
    df_chunk = fetch_fear_greed_history(limit=chunk_size, start=start)
    if df_chunk.empty:
        break
    dfs.append(df_chunk)
    if len(df_chunk) < chunk_size:
        break
    start += chunk_size

fear_greed = pd.concat(dfs).drop_duplicates(subset="timestamp").sort_values("timestamp")

print("✅ Fear & Greed Index downloaded:", len(fear_greed), "records")
display(fear_greed.head())


✅ Fear & Greed Index downloaded: 789 records


/tmp/ipython-input-3832929737.py:15: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
/tmp/ipython-input-3832929737.py:15: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")


timestamp  fear_greed_value value_classification
288 2023-06-29                59              Neutral
287 2023-06-30                59              Neutral
286 2023-07-01                60                Greed
285 2023-07-02                61                Greed
284 2023-07-03                61                Greed

In [ ]:
import time
import requests
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm

API_KEY = "b9d792ee-8769-44d9-ab43-109cd8973c29"
headers = {"X-CMC_PRO_API_KEY": API_KEY}

# ------------------------------
# Rate limiter wrapper
# ------------------------------
def safe_request(url, params):
    while True:
        r = requests.get(url, headers=headers, params=params)
        data = r.json()
        if "status" in data and data["status"]["error_code"] == 1008:
            print("⏳ Hit rate limit. Sleeping 60s...")
            time.sleep(60)   # wait a full minute
            continue
        return data

# ------------------------------
# Example: Get top 100 per day
# ------------------------------
def get_top100_for_day(date):
    url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/historical"
    params = {"date": date.strftime("%Y-%m-%d"), "limit": 100, "convert": "USD"}
    data = safe_request(url, params)
    if "data" not in data:
        return pd.DataFrame()
    return pd.DataFrame(data["data"])

# ------------------------------
# Loop with throttling
# ------------------------------
END_DATE = datetime.today()
START_DATE = END_DATE - timedelta(days=730)  # small test range

dates = [START_DATE + timedelta(days=i) for i in range((END_DATE-START_DATE).days+1)]
results = []

for d in tqdm(dates):
    df = get_top100_for_day(d)
    if not df.empty:
        df["date"] = d
        results.append(df)
    time.sleep(2)  # <-- throttle so you don't trigger 1008

if results:
    df_all = pd.concat(results, ignore_index=True)
    print("✅ Final shape:", df_all.shape)
else:
    print("⚠️ No data collected")


100%|██████████| 731/731 [26:24<00:00,  2.17s/it]

✅ Final shape: (3100, 19)


In [ ]:
import time
import requests
import pandas as pd
from datetime import datetime, timedelta

API_KEY = "b9d792ee-8769-44d9-ab43-109cd8973c29"
headers = {"X-CMC_PRO_API_KEY": API_KEY}

def safe_request(url, params):
    while True:
        r = requests.get(url, headers=headers, params=params)
        data = r.json()
        if "status" in data and data["status"]["error_code"] == 1008:
            print("⏳ Hit rate limit. Sleeping 60s...")
            time.sleep(60)
            continue
        return data

def test_single_date(date):
    url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/historical"
    params = {"date": date.strftime("%Y-%m-%d"), "limit": 10, "convert": "USD"}  # Smaller limit for testing
    data = safe_request(url, params)

    if "data" in data and data["data"]:
        return True
    return False

def find_historical_limit():
    """Binary search to find the exact historical data limit"""
    today = datetime.today()

    # Start with a range we know: 10 days works, 60 days doesn't
    min_days = 10   # We know this works
    max_days = 60   # We know this doesn't work

    print("🔍 Finding your exact historical data limit...")

    while max_days - min_days > 1:
        mid_days = (min_days + max_days) // 2
        test_date = today - timedelta(days=mid_days)

        print(f"Testing {mid_days} days ago ({test_date.strftime('%Y-%m-%d')})...")

        if test_single_date(test_date):
            print(f"✅ {mid_days} days works")
            min_days = mid_days
        else:
            print(f"❌ {mid_days} days doesn't work")
            max_days = mid_days

        time.sleep(1)  # Be nice to the API

    print(f"\n🎯 Your historical data limit is approximately {min_days} days")
    print(f"📅 You can safely collect data from {(today - timedelta(days=min_days)).strftime('%Y-%m-%d')} to today")

    return min_days

# Find your exact limit
limit_days = find_historical_limit()

# Update your original code with the correct date range
print(f"\n💡 Update your code with:")
print(f"START_DATE = END_DATE - timedelta(days={limit_days})")

🔍 Finding your exact historical data limit...
Testing 35 days ago (2025-07-31)...
❌ 35 days doesn't work
Testing 22 days ago (2025-08-13)...
✅ 22 days works
Testing 28 days ago (2025-08-07)...
✅ 28 days works
Testing 31 days ago (2025-08-04)...
✅ 31 days works
Testing 33 days ago (2025-08-02)...
❌ 33 days doesn't work
Testing 32 days ago (2025-08-03)...
❌ 32 days doesn't work

🎯 Your historical data limit is approximately 31 days
📅 You can safely collect data from 2025-08-04 to today

💡 Update your code with:
START_DATE = END_DATE - timedelta(days=31)


In [ ]:
# ------------------------------
# Save and download
# ------------------------------
if results:
    df_all = pd.concat(results, ignore_index=True)
    print("✅ Final shape:", df_all.shape)

    # Save to CSV
    filename = f"top100_{START_DATE.strftime('%Y-%m-%d')}_to_{END_DATE.strftime('%Y-%m-%d')}.csv"
    df_all.to_csv(filename, index=False)

    # Download to your computer (if using Colab)
    from google.colab import files
    files.download(filename)

else:
    print("⚠️ No data collected")


✅ Final shape: (3100, 19)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ===============================
# Step 6. Loop over top 100 coins
# ===============================
# Install tqdm if not already
!pip install tqdm

# Import tqdm notebook version
from tqdm.notebook import tqdm

end_date = datetime.today()
start_date = end_date - timedelta(days=3*365)  # last 3 years
all_data = []

for idx, row in tqdm(top_100.iterrows(), total=len(top_100)):
    coin_id = row["id"]
    symbol = row["symbol"]

    try:
        df = fetch_historical_data(coin_id, start_date, end_date)
        if not df.empty:
            df["symbol"] = symbol
            all_data.append(df)
            print(f"✅ Got data for {symbol}")
        else:
            print(f"⚠️ No data for {symbol}")
    except Exception as e:
        print(f"❌ Error fetching {symbol}: {e}")

    time.sleep(0.2)  # avoid rate limits

crypto_data = pd.concat(all_data, ignore_index=True)
print("✅ Historical data collected for top 100 coins")
display(crypto_data.head())

  0%|          | 0/100 [00:00<?, ?it/s]

⚠️ No data for coin_id 1
⚠️ No data for BTC
⚠️ No data for coin_id 1027
⚠️ No data for ETH
⚠️ No data for coin_id 52
⚠️ No data for XRP
⚠️ No data for coin_id 825
⚠️ No data for USDT
⚠️ No data for coin_id 1839
⚠️ No data for BNB
⚠️ No data for coin_id 5426
⚠️ No data for SOL
⚠️ No data for coin_id 3408
⚠️ No data for USDC
⚠️ No data for coin_id 1958
⚠️ No data for TRX
⚠️ No data for coin_id 74
⚠️ No data for DOGE
⚠️ No data for coin_id 2010
⚠️ No data for ADA
⚠️ No data for coin_id 1975
⚠️ No data for LINK
⚠️ No data for coin_id 32196
⚠️ No data for HYPE
⚠️ No data for coin_id 512
⚠️ No data for XLM
⚠️ No data for coin_id 29470
⚠️ No data for USDe
⚠️ No data for coin_id 20947
⚠️ No data for SUI
⚠️ No data for coin_id 1831
⚠️ No data for BCH
⚠️ No data for coin_id 4642
⚠️ No data for HBAR
⚠️ No data for coin_id 5805
⚠️ No data for AVAX
⚠️ No data for coin_id 3957
⚠️ No data for LEO
⚠️ No data for coin_id 2
⚠️ No data for LTC
⚠️ No data for coin_id 11419
⚠️ No data for TON
⚠️ No data fo

ValueError: No objects to concatenate

In [ ]:
from google.colab import files

files.download("crypto_data.csv")
files.download("fear_greed.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ===============================
# Step 7. Merge with Fear & Greed
# ===============================
crypto_data = pd.concat(all_data, ignore_index=True)
crypto_data.head()


MergeError: Not allowed to merge between different levels. (2 levels on the left, 1 on the right)

In [ ]:
# ===============================
# Step 8. Save & Download
# ===============================
merged.to_csv("crypto_top100_with_feargreed.csv", index=False)
print("✅ Dataset saved as crypto_top100_with_feargreed.csv")

files.download("crypto_top100_with_feargreed.csv")

In [ ]:
import time
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone

API_KEY = "b9d792ee-8769-44d9-ab43-109cd8973c29"
headers = {"X-CMC_PRO_API_KEY": API_KEY}

# --- Safe request to handle rate limits ---
def safe_request(url, params):
    while True:
        r = requests.get(url, headers=headers, params=params)
        data = r.json()
        # Error code 1008 = rate limit
        if "status" in data and data["status"]["error_code"] == 1008:
            print("⏳ Hit rate limit. Sleeping 60s...")
            time.sleep(60)
            continue
        return data

# --- Collect top 100 coins for each day in date range ---
def fetch_top_100_last_n_days(n_days=31):
    all_coins = set()
    today = datetime.now(timezone.utc)
    start_date = today - timedelta(days=n_days)

    for i in range(n_days):
        date = start_date + timedelta(days=i)
        params = {
            "date": date.strftime("%Y-%m-%d"),
            "limit": 100,
            "convert": "USD"
        }
        url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/historical"
        data = safe_request(url, params)

        if "data" in data and isinstance(data["data"], list):
            for coin in data["data"]:
                all_coins.add((coin["name"], coin["symbol"]))

        print(f"✅ Processed {date.strftime('%Y-%m-%d')}")
        time.sleep(1)  # Be kind to API

    return all_coins

# --- Fetch and save ---
top_coins = fetch_top_100_last_n_days(n_days=31)
df = pd.DataFrame(list(top_coins), columns=["Name", "Symbol"])
df.sort_values("Name", inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_csv("top_100_coins_last_31_days.csv", index=False)

print(f"\n🎉 Saved {len(df)} unique coins to CSV!")
df.head()


✅ Processed 2025-08-04
✅ Processed 2025-08-05
✅ Processed 2025-08-06
✅ Processed 2025-08-07
✅ Processed 2025-08-08
✅ Processed 2025-08-09
✅ Processed 2025-08-10
✅ Processed 2025-08-11
✅ Processed 2025-08-12
✅ Processed 2025-08-13
✅ Processed 2025-08-14
✅ Processed 2025-08-15
✅ Processed 2025-08-16
✅ Processed 2025-08-17
✅ Processed 2025-08-18
✅ Processed 2025-08-19
✅ Processed 2025-08-20
✅ Processed 2025-08-21
✅ Processed 2025-08-22
✅ Processed 2025-08-23
✅ Processed 2025-08-24
✅ Processed 2025-08-25
✅ Processed 2025-08-26
✅ Processed 2025-08-27
✅ Processed 2025-08-28
✅ Processed 2025-08-29
✅ Processed 2025-08-30
✅ Processed 2025-08-31
✅ Processed 2025-09-01
✅ Processed 2025-09-02
✅ Processed 2025-09-03

🎉 Saved 109 unique coins to CSV!


Name Symbol
0                 AB     AB
1               Aave   AAVE
2  Aerodrome Finance   AERO
3           Algorand   ALGO
4              Aptos    APT

In [ ]:
import time
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone

# For download link in Google Colab
from google.colab import files  # Only needed if using Colab

API_KEY = "b9d792ee-8769-44d9-ab43-109cd8973c29"
headers = {"X-CMC_PRO_API_KEY": API_KEY}

# --- Safe request to handle rate limits ---
def safe_request(url, params):
    while True:
        r = requests.get(url, headers=headers, params=params)
        data = r.json()
        if "status" in data and data["status"]["error_code"] == 1008:
            print("⏳ Hit rate limit. Sleeping 60s...")
            time.sleep(60)
            continue
        return data

# --- Collect top 100 coins for each day in date range ---
def fetch_top_100_last_n_days(n_days=31):
    all_coins = set()
    today = datetime.now(timezone.utc)
    start_date = today - timedelta(days=n_days)

    for i in range(n_days):
        date = start_date + timedelta(days=i)
        params = {
            "date": date.strftime("%Y-%m-%d"),
            "limit": 100,
            "convert": "USD"
        }
        url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/historical"
        data = safe_request(url, params)

        if "data" in data and isinstance(data["data"], list):
            for coin in data["data"]:
                all_coins.add((coin["name"], coin["symbol"]))

        print(f"✅ Processed {date.strftime('%Y-%m-%d')}")
        time.sleep(1)

    return all_coins

# --- Fetch data ---
top_coins = fetch_top_100_last_n_days(n_days=31)
df = pd.DataFrame(list(top_coins), columns=["Name", "Symbol"])
df.sort_values("Name", inplace=True)
df.reset_index(drop=True, inplace=True)

# --- Save & Download ---
csv_filename = "top_100_coins_last_31_days.csv"
df.to_csv(csv_filename, index=False)
print(f"✅ Dataset saved as {csv_filename}")

# Download in Colab
files.download(csv_filename)


✅ Processed 2025-08-04
✅ Processed 2025-08-05
✅ Processed 2025-08-06
✅ Processed 2025-08-07
✅ Processed 2025-08-08
✅ Processed 2025-08-09
✅ Processed 2025-08-10
✅ Processed 2025-08-11
✅ Processed 2025-08-12
✅ Processed 2025-08-13
✅ Processed 2025-08-14
✅ Processed 2025-08-15
✅ Processed 2025-08-16
✅ Processed 2025-08-17
✅ Processed 2025-08-18
✅ Processed 2025-08-19
✅ Processed 2025-08-20
✅ Processed 2025-08-21
✅ Processed 2025-08-22
✅ Processed 2025-08-23
✅ Processed 2025-08-24
✅ Processed 2025-08-25
✅ Processed 2025-08-26
✅ Processed 2025-08-27
✅ Processed 2025-08-28
✅ Processed 2025-08-29
✅ Processed 2025-08-30
✅ Processed 2025-08-31
✅ Processed 2025-09-01
✅ Processed 2025-09-02
✅ Processed 2025-09-03
✅ Dataset saved as top_100_coins_last_31_days.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>